In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import struct
from skimage import io,data

data = []
model = []

class MnistData:
    def __init__(self, train_image_path, train_label_path,
                 test_image_path, test_label_path, data_increament):
        # 训练集和验证集的文件路径
        self.train_image_path = train_image_path
        self.train_label_path = train_label_path
        self.test_image_path = test_image_path
        self.test_label_path = test_label_path
        self.data_increament = data_increament

        # 获取训练集和验证集数据
        # get_data()方法，参数为0获取训练集数据，参数为1获取验证集
        self.train_images, self.train_labels = self.get_data(0)
        self.test_images, self.test_labels = self.get_data(1)

    def get_data(self, data_type):
        if data_type == 0:  # 获取训练集数据
            image_path = self.train_image_path
            label_path = self.train_label_path
        else:  # 获取验证集数据
            image_path = self.test_image_path
            label_path = self.test_label_path

        with open(image_path, 'rb') as file1:
            image_file = file1.read()
        with open(label_path, 'rb') as file2:
            label_file = file2.read()

        label_index = 0
        image_index = 0
        labels = []
        images = []
        images_noise = []


        # 读取训练集图像数据文件的文件信息
        magic, num_of_datasets, rows, columns = struct.unpack_from('>IIII',
                                        image_file, image_index)
        image_index += struct.calcsize('>IIII')

        for i in range(num_of_datasets):
            # 读取784个unsigned byte，即一副图像的所有像素值
            temp = struct.unpack_from('>784B', image_file, image_index)
            # 将读取的像素数据转换成28*28的矩阵
            temp = np.reshape(temp, (28, 28))
            # 归一化处理
            temp = temp / 255
            images.append(temp)

            if self.data_increament == True:
              if data_type == 0:
                rows, cols = 28, 28
                buf = np.empty(shape=[28, 28])
                for x in range(28):    
                  for y in range(28):   
                    buf[x, y] = temp[x, y] + np.random.randint(-30,30)/100
                    if buf[x, y] < 0:
                      buf[x, y] = 0
                    if buf[x, y] > 1:
                      buf[x, y] = 1
                images_noise.append(buf)

            image_index += struct.calcsize('>784B')  # 每次增加784B

        # 跳过描述信息
        label_index += struct.calcsize('>II')
        labels = struct.unpack_from('>' + str(num_of_datasets) + 'B', label_file, label_index)

        # one-hot
        labels = np.eye(10)[np.array(labels)]
        # 扩充数据集后，类标也需要相应扩充
        if self.data_increament == True:
          if data_type == 0:
            labels = np.concatenate((labels, labels))
            images = np.concatenate((images, images_noise))

        return np.array(images), labels


In [ ]:
# MINIST
train_image_path = '/content/drive/My Drive/data/train-images.idx3-ubyte'
train_label_path = '/content/drive/My Drive/data/train-labels.idx1-ubyte'
test_image_path = '/content/drive/My Drive/data/t10k-images.idx3-ubyte'
test_label_path = '/content/drive/My Drive/data/t10k-labels.idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path, data_increament = False)

data.train_images = np.expand_dims(data.train_images, axis=3)
data.test_images = np.expand_dims(data.test_images, axis=3)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [3]:
# Fashion MINIST
train_image_path = '/content/drive/My Drive/fasion_data/train-images-idx3-ubyte'
train_label_path = '/content/drive/My Drive/fasion_data/train-labels-idx1-ubyte'
test_image_path = '/content/drive/My Drive/fasion_data/t10k-images-idx3-ubyte'
test_label_path = '/content/drive/My Drive/fasion_data/t10k-labels-idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path, data_increament = False)

data.train_images = np.expand_dims(data.train_images, axis=3)
data.test_images = np.expand_dims(data.test_images, axis=3)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
# Fashion MINIST (data increament)
train_image_path = '/content/drive/My Drive/fasion_data/train-images-idx3-ubyte'
train_label_path = '/content/drive/My Drive/fasion_data/train-labels-idx1-ubyte'
test_image_path = '/content/drive/My Drive/fasion_data/t10k-images-idx3-ubyte'
test_label_path = '/content/drive/My Drive/fasion_data/t10k-labels-idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path, data_increament = True)

data.train_images = np.expand_dims(data.train_images, axis=3)
data.test_images = np.expand_dims(data.test_images, axis=3)

print(type(data.train_images))
print(type(data.train_labels))

In [ ]:

print(type(data.train_images))
print(type(data.train_labels))

print(type(data.train_images[0]))
print(type(data.train_labels[0]))

print(data.train_images[0])
print(data.train_labels[0])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(data.train_images[2].reshape(28, 28), cmap='gray')

In [ ]:
print("train.shpae test.shape", data.train_images.shape, data.test_images.shape)
print("train.shpae test.shape", data.train_labels.shape, data.test_labels.shape)

train.shpae test.shape (60000, 28, 28, 1) (10000, 28, 28, 1)
train.shpae test.shape (60000, 10) (10000, 10)


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

# Define model
def get_model():
    # Build a sequential model
    model = tf.keras.Sequential()

    model.add(layers.InputLayer(input_shape=(28, 28, 1)))
    # Nomalization
    model.add(layers.BatchNormalization())

    # First convolutional layer
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(layers.Dropout(0.3))

    # Second convolutional layer
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
    model.add(layers.Dropout(0.3))

    # Convert the output feature mapping of the previous layer into one-dimensional data
    model.add(layers.Flatten())

    # First fully connected layer
    model.add(layers.Dense(256, activation = 'relu'))
    model.add(layers.Dropout(0.4))

    # Second fully connected layer
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dropout(0.3))

    # Nomalization
    model.add(layers.BatchNormalization())

    # Third fully connected layer
    model.add(layers.Dense(10, activation = 'softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    
    # Print model
    model.summary()

    return model

model = get_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 128)       0

In [ ]:
model.fit(data.train_images, data.train_labels, epochs=50, batch_size=500)

In [ ]:
model.evaluate(data.test_images, data.test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 0.3532 - accuracy: 0.9295


[0.35324764251708984, 0.9294999837875366]

In [ ]:
model.fit(data.test_images, data.test_labels, epochs=10)